# IMBALANCED DATA

In [1]:
!pip install pandas-datareader

In [2]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action="ignore")

* **Create a logistic regression model and measure the performance of it.**

In [3]:
df = pd.read_csv('data/cleveland-0_vs_4.dat',skiprows = 18,error_bad_lines=False)
df.columns =['age', 'sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','num'] 
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,negative
1,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,negative
2,56.0,1.0,2.0,120.0,236.0,0.0,0.0,178.0,0.0,0.8,1.0,0.0,3.0,negative
3,57.0,0.0,4.0,120.0,354.0,0.0,0.0,163.0,1.0,0.6,1.0,0.0,3.0,negative
4,57.0,1.0,4.0,140.0,192.0,0.0,0.0,148.0,0.0,0.4,2.0,0.0,6.0,negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,64.0,1.0,4.0,145.0,212.0,0.0,2.0,132.0,0.0,2.0,2.0,2.0,6.0,positive
172,38.0,1.0,1.0,120.0,231.0,0.0,0.0,182.0,1.0,3.8,2.0,0.0,7.0,positive
173,61.0,1.0,4.0,138.0,166.0,0.0,2.0,125.0,1.0,3.6,2.0,1.0,3.0,positive
174,58.0,1.0,4.0,114.0,318.0,0.0,1.0,140.0,0.0,4.4,3.0,3.0,6.0,positive


In [4]:
df.isnull().sum()*100/df.shape[0]

age         0.0
sex         0.0
cp          0.0
trestbps    0.0
chol        0.0
fbs         0.0
restecg     0.0
thalach     0.0
exang       0.0
oldpeak     0.0
slope       0.0
ca          0.0
thal        0.0
num         0.0
dtype: float64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       176 non-null    float64
 1   sex       176 non-null    float64
 2   cp        176 non-null    float64
 3   trestbps  176 non-null    float64
 4   chol      176 non-null    float64
 5   fbs       176 non-null    float64
 6   restecg   176 non-null    float64
 7   thalach   176 non-null    float64
 8   exang     176 non-null    float64
 9   oldpeak   176 non-null    float64
 10  slope     176 non-null    float64
 11  ca        176 non-null    object 
 12  thal      176 non-null    object 
 13  num       176 non-null    object 
dtypes: float64(11), object(3)
memory usage: 19.4+ KB


In [6]:
df.ca.value_counts()

0.0       132
1.0        24
2.0         9
3.0         8
<null>      3
Name: ca, dtype: int64

In [7]:
df.thal[df[df.thal == "<null>"].index] ="0.0"

In [8]:
df.ca.value_counts()

0.0       132
1.0        24
2.0         9
3.0         8
<null>      3
Name: ca, dtype: int64

In [9]:
df.ca[df[df.ca == "<null>"].index] ="0.0"

In [10]:
df['ca'] = df['ca'].astype(float)
df['thal'] = df['thal'].astype(float)

In [11]:
dummy = pd.get_dummies(df.num)
df["is_positive"] = dummy.positive
X = df.drop(["num","is_positive"], axis=1)
y= df.is_positive

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [29]:
from sklearn.linear_model import LinearRegression,LogisticRegression

model_one = LogisticRegression()
model_one.fit(X_train,y_train)
ytest_score_one = model_one.score(X_test,y_test)
ytrain_score_one = model_one.score(X_train,y_train)

print("One vs. Rest : \n -------------- \n Test accuracy : {:.2f} \n Train accuracy : {:.2f}".format(ytest_score_one,ytrain_score_one))

One vs. Rest : 
 -------------- 
 Test accuracy : 0.94 
 Train accuracy : 0.97


In [28]:
model_softmax = LogisticRegression(multi_class="multinomial")
model_softmax.fit(X_train,y_train)
ytest_score_soft = model_softmax.score(X_test,y_test)
ytrain_score_sof = model_softmax.score(X_train,y_train)

print("Softmax : \n ---------------- \n Test accuracy : {:.2f} \n Train accuracy : {:.2f}".format(ytest_score_soft,ytrain_score_sof))

Softmax : 
 ---------------- 
 Test accuracy : 0.94 
 Train accuracy : 0.98


In [38]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,classification_report

ytest_pred_softmax = model_softmax.predict(X_test)
conf_test_soft = confusion_matrix(y_test,ytest_pred_softmax)
print(conf_test_soft)

[[57  0]
 [19 32]]


In [31]:
ytrain_pred_softmax = model_one.predict(X_train)
conf_train_soft = confusion_matrix(y_train,ytrain_pred_softmax)
print(conf_train_soft)

[[130   0]
 [  4   6]]


* *We might understand accuracy is very good for our model but When we looked the confussion matrix we have just 10 positive estimate and 4 of them actually is wrong*

* **By experimenting with different methods and class ratios; overcome class imbalance, determine the best performing method and class ratio.**

In [34]:
!pip install -U imbalanced-learn

Requirement already up-to-date: imbalanced-learn in c:\users\cemce\anaconda3\lib\site-packages (0.7.0)


In [32]:
from imblearn.over_sampling import SMOTE,ADASYN

X = df.drop(["num","is_positive"], axis=1)
y= df.is_positive

sm = SMOTE(random_state=27, sampling_strategy=1.0)
X_smote, y_smote = sm.fit_sample(X, y)

In [43]:
def model_metrics(X,y):    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    logreg_softmax = LogisticRegression(multi_class="multinomial")
    logreg_softmax.fit(X_train,y_train)

    test_pred_soft = logreg_softmax.predict(X_test)
    train_pred_soft = logreg_softmax.predict(X_train)

    print("Softmax: \n -----------------\n")

    test_score_soft = accuracy_score(y_test,test_pred_soft)
    train_score_soft = accuracy_score(y_train,train_pred_soft)
    print("Accuracy Scores : \n -------------\n Test Score : {} \n Train Score : {}".format(test_score_soft,train_score_soft))


    test_conf_soft = confusion_matrix(y_test,test_pred_soft)
    train_conf_soft = confusion_matrix(y_train,train_pred_soft)
    print("Confussion matrix : \n ---------------\n Test Confussion matrix : \n {} \n Train Confussion matrix : \n {}".format(test_conf_soft,train_conf_soft))

    print("Train dataset : ")
    print(classification_report(y_train,train_pred_soft))
    print("Test dataset : ")
    print(classification_report(y_test,test_pred_soft))

In [44]:
model_metrics(X_smote,y_smote)

Softmax: 
 -----------------

Accuracy Scores : 
 -------------
 Test Score : 0.9351851851851852 
 Train Score : 0.9954128440366973
Confussion matrix : 
 ---------------
 Test Confussion matrix : 
 [[50  7]
 [ 0 51]] 
 Train Confussion matrix : 
 [[105   1]
 [  0 112]]
Train dataset : 
              precision    recall  f1-score   support

           0       1.00      0.99      1.00       106
           1       0.99      1.00      1.00       112

    accuracy                           1.00       218
   macro avg       1.00      1.00      1.00       218
weighted avg       1.00      1.00      1.00       218

Test dataset : 
              precision    recall  f1-score   support

           0       1.00      0.88      0.93        57
           1       0.88      1.00      0.94        51

    accuracy                           0.94       108
   macro avg       0.94      0.94      0.94       108
weighted avg       0.94      0.94      0.94       108



* *Adasyn*

In [46]:
X = df.drop(["num","is_positive"], axis=1)
y= df.is_positive

ad = ADASYN()
X_adasyn, y_adasyn = ad.fit_sample(X, y)

In [47]:
model_metrics(X_adasyn,y_adasyn)

Softmax: 
 -----------------

Accuracy Scores : 
 -------------
 Test Score : 0.9357798165137615 
 Train Score : 0.9908675799086758
Confussion matrix : 
 ---------------
 Test Confussion matrix : 
 [[50  7]
 [ 0 52]] 
 Train Confussion matrix : 
 [[104   2]
 [  0 113]]
Train dataset : 
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       106
           1       0.98      1.00      0.99       113

    accuracy                           0.99       219
   macro avg       0.99      0.99      0.99       219
weighted avg       0.99      0.99      0.99       219

Test dataset : 
              precision    recall  f1-score   support

           0       1.00      0.88      0.93        57
           1       0.88      1.00      0.94        52

    accuracy                           0.94       109
   macro avg       0.94      0.94      0.94       109
weighted avg       0.94      0.94      0.94       109



In [48]:
df.to_csv (r'data\heart_disease.csv', index = False, header=True)